In [ ]:
## Reference : https://python.langchain.com/docs/integrations/llms/huggingface_hub

## !pip install huggingface_hub

In [20]:
import pandas as pd
import numpy as np 

In [21]:
#Sample dataset that has jobs details
Jobs = pd.read_csv('./emscad_v1.csv')

In [73]:
Jobs[['title','company_profile','location','description']].head()

,title,company_profile,location,description
0,Marketing Intern,"<h3>We're Food52, and we've created a groundbr...","US, NY, New York","<p>Food52, a fast-growing, James Beard Award-w..."
1,Customer Service - Cloud Video Production,"<h3>90 Seconds, the worlds Cloud Video Product...","NZ, , Auckland",<p>Organised - Focused - Vibrant - Awesome!<br...
2,Commissioning Machinery Assistant (CMA),<h3></h3>\r\n<p>Valor Services provides Workfo...,"US, IA, Wever","<p>Our client, located in Houston, is actively..."
3,Account Executive - Washington DC,<p>Our passion for improving quality of life t...,"US, DC, Washington",<p><b>THE COMPANY: ESRI – Environmental System...
4,Bill Review Manager,<p>SpotSource Solutions LLC is a Global Human ...,"US, FL, Fort Worth",<p><b>JOB TITLE:</b> Itemization Review Manage...


In [74]:
#Sample dataset for demo get relevant columns
sample= Jobs[['title','description','location']].sample(50,random_state=1)
sample.head()

,title,description,location
5736,Jr. Business Analyst & Quality Analyst (entry...,<p>Duration: Full time / W2<br><br>Location: P...,"US, NJ, PISCATAWAY"
7106,English Teacher Abroad,"<p>Play with kids, get paid for it </p>\r\n<p>...","US, PA, Scranton"
11978,SQL Server Database Developer Job opportunity ...,<p><br><br>Position : SQL Server Database Deve...,"US, IL, Barrington"
9374,Legal Analyst - 12 Month FTC,<p><b>Description</b></p>\r\n<p>Our mission at...,"GB, LND, London"
1300,Part-Time Finance Assistant,<p>Salary:£9 - £10 per hour <br><br>We are cur...,"GB, LND,"


In [34]:
# Create a function remove HTML tags

from bs4 import BeautifulSoup

def remove_html_tags(text):
    soup = BeautifulSoup(text, 'html.parser')
    return soup.get_text()



In [75]:
#Clean Description Column
sample['description'] = sample['description'].apply(remove_html_tags)
sample.head()

,title,description,location
5736,Jr. Business Analyst & Quality Analyst (entry...,"Duration: Full time / W2Location: Piscataway,N...","US, NJ, PISCATAWAY"
7106,English Teacher Abroad,"Play with kids, get paid for it \nLove travel?...","US, PA, Scranton"
11978,SQL Server Database Developer Job opportunity ...,Position : SQL Server Database DeveloperJob Lo...,"US, IL, Barrington"
9374,Legal Analyst - 12 Month FTC,Description\nOur mission at MarketInvoice is t...,"GB, LND, London"
1300,Part-Time Finance Assistant,Salary:£9 - £10 per hour We are currently goin...,"GB, LND,"


### Transformers : Pipeline to scan through job descriptions

In [7]:
from transformers import pipeline
# Load the question-answering pipeline
qa_pipeline = pipeline('question-answering', model='deepset/roberta-base-squad2')


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

### Demo to see mechanism

In [70]:
def demo(input,questions):
    answers = []
    print("\n\n")
    print("-" * 30)
    for question in questions:
        answer = qa_pipeline(question=question, context=input)['answer']
        print(question,': ', answer)

    


In [71]:
#Example: #context = 'Title: Exciting Work-from-Home Opportunity! Earn Big Money Fast! Description: Are you tired of your 9-5 job and looking for a change? We have the perfect opportunity for you! Join our exclusive team and start making money from the comfort of your own home. Key Benefits: No Experience Required! Work Flexible Hours, Part-Time or Full-Time! Earn Up to $10,000 per Month! Quick and Easy Payments via Bitcoin! Requirements: Must be 18 years or older. No formal education needed. No background checks or verifications. Work-at-home kit available for a small fee. Perks: Free Accommodation Options! Travel Opportunities! Exciting Bonuses and Incentives! To apply, simply click the link below and register now! Dont miss out on this once-in-a-lifetime opportunity to change your life! SpammyLink123.com/ApplyNow Note: This is a fictional example and not a real job opportunity. Actual spam job ads may use deceptive language, false promises, or requests for personal information. Always be cautious and thoroughly research any job opportunity before providing personal or financial information'
#questions = ["Can minors be employed here", "Does this provide accomodation", "Does this have educational requirements", "What is the salary range offered in dollars per year format"]
 
text_to_search = input("Enter the text to search: ")
questions = input("Enter the questions (comma-separated): ").split(',')

demo(text_to_search,questions)

Enter the text to search: Title: Exciting Work-from-Home Opportunity! Earn Big Money Fast! Description: Are you tired of your 9-5 job and looking for a change? We have the perfect opportunity for you! Join our exclusive team and start making money from the comfort of your own home. Key Benefits: No Experience Required! Work Flexible Hours, Part-Time or Full-Time! Earn Up to $10,000 per Month! Quick and Easy Payments via Bitcoin! Requirements: Must be 18 years or older. No formal education needed. No background checks or verifications. Work-at-home kit available for a small fee. Perks: Free Accommodation Options! Travel Opportunities! Exciting Bonuses and Incentives! To apply, simply click the link below and register now! Dont miss out on this once-in-a-lifetime opportunity to change your life! SpammyLink123.com/ApplyNow Note: This is a fictional example and not a real job opportunity. Actual spam job ads may use deceptive language, false promises, or requests for personal information. 

In [32]:
#Demo of how each snippet works

# User inputs
context = input("Enter the text to search: ")
questions = input("Enter the questions (comma-separated): ").split(',')

#context = 'Title: Exciting Work-from-Home Opportunity! Earn Big Money Fast! Description: Are you tired of your 9-5 job and looking for a change? We have the perfect opportunity for you! Join our exclusive team and start making money from the comfort of your own home. Key Benefits: No Experience Required! Work Flexible Hours, Part-Time or Full-Time! Earn Up to $10,000 per Month! Quick and Easy Payments via Bitcoin! Requirements: Must be 18 years or older. No formal education needed. No background checks or verifications. Work-at-home kit available for a small fee. Perks: Free Accommodation Options! Travel Opportunities! Exciting Bonuses and Incentives! To apply, simply click the link below and register now! Dont miss out on this once-in-a-lifetime opportunity to change your life! SpammyLink123.com/ApplyNow Note: This is a fictional example and not a real job opportunity. Actual spam job ads may use deceptive language, false promises, or requests for personal information. Always be cautious and thoroughly research any job opportunity before providing personal or financial information'
#questions = ["Can minors be employed here", "Does this provide accomodation", "Does this have educational requirements", "What is the salary range offered in dollars per year format"]
answers = []

for question in questions:
        answer = qa_pipeline(question=question, context=context)['answer']
        answers.append(answer)
        
print(answers)

['Must be 18 years or older', 'Free Accommodation Options', 'No formal education needed', '$10,000 per Month']


### Scaling this to scraped job descriptions

In [54]:
def answer_question(row,questions):
    context = row['description']
    answers = []
    for question in questions:
        answer = qa_pipeline(question=question, context=context)['answer']
        answers.append(answer)

    return pd.Series(answers, index=questions)


In [59]:
#Try with sample 
questions = ["Can minors be employed here", "Does this provide accomodation", "Does this have educational requirements", "What is the salary range offered in dollars per year format"]

sample[questions] = sample.apply(lambda row: answer_question(row, questions), axis=1)


In [60]:
sample.head()

,title,description,location,fraudulent,Can minors be employed here,Does this provide accomodation,Does this have educational requirements,What is the salary range offered in dollars per year format
5736,Jr. Business Analyst & Quality Analyst (entry...,"Duration: Full time / W2Location: Piscataway,N...","US, NJ, PISCATAWAY",f,entry level candidates with BA/QA skills are e...,entry level candidates with BA/QA skills are e...,"Masters Degree in Computer Science, Engineerin...",Expected Rate / Salary :\n
7106,English Teacher Abroad,"Play with kids, get paid for it \nLove travel?...","US, PA, Scranton",f,\n12 month contract : Apply today,Housing provided,\n12 month contract : Apply today,"$1,500+ USD monthly"
11978,SQL Server Database Developer Job opportunity ...,Position : SQL Server Database DeveloperJob Lo...,"US, IL, Barrington",f,Candidates should understand and have a proven...,Must be able to manage database objects and da...,BS degree,H1B / EAD
9374,Legal Analyst - 12 Month FTC,Description\nOur mission at MarketInvoice is t...,"GB, LND, London",f,Legal Analyst,This is a unique and very fulfilling role for ...,You will utilise your considerable academic an...,Legal Analyst
1300,Part-Time Finance Assistant,Salary:£9 - £10 per hour We are currently goin...,"GB, LND,",f,periods of time when you will be working un-su...,"You will offer a flexible, adaptable approach",periods of time when you will be working un-su...,:£9 - £10 per hour


### Named Entity Recogntion on the flagged posts 

In [62]:
# Load the NER pipeline
ner_pipeline = pipeline('ner', model='dbmdz/bert-large-cased-finetuned-conll03-english')

def extract_entities(text):
    entities = ner_pipeline(text)
    entity_dict = {'name': [], 'org': [], 'gpe': []}

    for ent in entities:
        entity_type = ent['entity']
        entity_text = ent['word']

        if entity_type in ('B-PER', 'I-PER'):
            entity_dict['name'].append(entity_text)
        elif entity_type in ('B-ORG', 'I-ORG'):
            entity_dict['org'].append(entity_text)
        elif entity_type in ('B-GPE', 'I-GPE'):
            entity_dict['gpe'].append(entity_text)

    return pd.Series(entity_dict)


Some weights of the model checkpoint at dbmdz/bert-large-cased-finetuned-conll03-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [66]:
# Apply the function to extract entities and create new columns
sample[['name', 'org', 'gpe']] = sample['description'].apply(extract_entities)

# Print the resulting DataFrame
sample.head()

,title,description,location,fraudulent,Can minors be employed here,Does this provide accomodation,Does this have educational requirements,What is the salary range offered in dollars per year format,name,org,gpe
5736,Jr. Business Analyst & Quality Analyst (entry...,"Duration: Full time / W2Location: Piscataway,N...","US, NJ, PISCATAWAY",f,entry level candidates with BA/QA skills are e...,entry level candidates with BA/QA skills are e...,"Masters Degree in Computer Science, Engineerin...",Expected Rate / Salary :\n,[],"[##ta, ##way, Engineering, Oracle, ##QL]",[]
7106,English Teacher Abroad,"Play with kids, get paid for it \nLove travel?...","US, PA, Scranton",f,\n12 month contract : Apply today,Housing provided,\n12 month contract : Apply today,"$1,500+ USD monthly","[Gabriel, Ad, ##kins]",[],[]
11978,SQL Server Database Developer Job opportunity ...,Position : SQL Server Database DeveloperJob Lo...,"US, IL, Barrington",f,Candidates should understand and have a proven...,Must be able to manage database objects and da...,BS degree,H1B / EAD,[],"[##QL, S, ##QL, Analysis, Services, Report, Se...",[]
9374,Legal Analyst - 12 Month FTC,Description\nOur mission at MarketInvoice is t...,"GB, LND, London",f,Legal Analyst,This is a unique and very fulfilling role for ...,You will utilise your considerable academic an...,Legal Analyst,[],"[Market, ##I, ##n, ##vo, ##ice, Market, ##I, #...",[]
1300,Part-Time Finance Assistant,Salary:£9 - £10 per hour We are currently goin...,"GB, LND,",f,periods of time when you will be working un-su...,"You will offer a flexible, adaptable approach",periods of time when you will be working un-su...,:£9 - £10 per hour,[],[],[]


In [67]:
sample['description']

5736     Duration: Full time / W2Location: Piscataway,N...
7106     Play with kids, get paid for it \nLove travel?...
11978    Position : SQL Server Database DeveloperJob Lo...
9374     Description\nOur mission at MarketInvoice is t...
1300     Salary:£9 - £10 per hour We are currently goin...
Name: description, dtype: object

In [ ]:
sample[['Country', 'State', 'City']] = sample['location'].str.split(',', expand=True)
sample.head()

In [44]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

# Load pre-trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Function to generate a response based on the input text
def generate_response(input_text, max_length=150):
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    # Ensure that max_length is less than or equal to the length of input_ids
    max_length = min(max_length, input_ids.size(1))
    outputs = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# User inputs
text_to_search = input("Enter the text to search: ")
questions = input("Enter the questions (comma-separated): ").split(',')

# Generate response for each question
responses = []
for question in questions:
    input_text = f"{text_to_search} Question: {question}"

    # Split input text into smaller chunks (e.g., 150 tokens per chunk)
    chunk_size = 150
    chunks = [input_text[i:i+chunk_size] for i in range(0, len(input_text), chunk_size)]

    # Generate response for each chunk and concatenate the results
    chunk_responses = [generate_response(chunk) for chunk in chunks]
    responses.append(' '.join(chunk_responses))

# Create a DataFrame with the results
df = pd.DataFrame({'Question': questions, 'Response': responses})

# Parse the generated responses into a categorical column (example: Positive/Negative)
df['Category'] = df['Response'].apply(lambda x: 'Positive' if 'positive keyword' in x else 'Negative')

# Print the DataFrame
print(df)


Enter the text to search: Title: Exciting Work-from-Home Opportunity! Earn Big Money Fast! Description: Are you tired of your 9-5 job and looking for a change? We have the perfect opportunity for you! Join our exclusive team and start making money from the comfort of your own home. Key Benefits: No Experience Required! Work Flexible Hours, Part-Time or Full-Time! Earn Up to $10,000 per Month! Quick and Easy Payments via Bitcoin! Requirements: Must be 18 years or older. No formal education needed. No background checks or verifications. Work-at-home kit available for a small fee. Perks: Free Accommodation Options! Travel Opportunities! Exciting Bonuses and Incentives! To apply, simply click the link below and register now! Dont miss out on this once-in-a-lifetime opportunity to change your life! SpammyLink123.com/ApplyNow Note: This is a fictional example and not a real job opportunity. Actual spam job ads may use deceptive language, false promises, or requests for personal information. 

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\Apoorva Shetty\anaconda3\lib\site-packages\transformers\generation\utils.py:1281: UserWarning: Input length of input_ids is 150, but `max_length` is set to 150. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


RuntimeError: The expanded size of the tensor (150) must match the existing size (151) at non-singleton dimension 0.  Target sizes: [150].  Tensor sizes: [151]

In [45]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import pandas as pd

# Load pre-trained model and tokenizer
model = GPT2LMHeadModel.from_pretrained("gpt2")
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

# Function to generate a response based on the input text
def generate_response(input_text, max_length=150):
    input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=max_length, truncation=True)
    # Ensure that max_length is less than or equal to the length of input_ids
    max_length = min(max_length, input_ids.size(1))
    outputs = model.generate(input_ids, max_length=max_length, num_beams=5, no_repeat_ngram_size=2, top_k=50, top_p=0.95, temperature=0.7)
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return response

# User inputs
text_to_search = input("Enter the text to search: ")
questions = input("Enter the questions (comma-separated): ").split(',')

# Generate response for each question
responses = []
for question in questions:
    input_text = f"{text_to_search} Question: {question}"

    # Split input text into smaller chunks (e.g., 150 tokens per chunk)
    chunk_size = 150
    chunks = [input_text[i:i+chunk_size] for i in range(0, len(input_text), chunk_size)]

    # Generate response for each chunk and concatenate the results
    chunk_responses = [generate_response(chunk) for chunk in chunks]
    responses.append(' '.join(chunk_responses))

# Create a DataFrame with the results
df = pd.DataFrame({'Question': questions, 'Response': responses})

# Parse the generated responses into a categorical column (example: Positive/Negative)
df['Category'] = df['Response'].apply(lambda x: 'Positive' if 'positive keyword' in x else 'Negative')

# Print the DataFrame
print(df)


Enter the text to search: Exciting Work-from-Home Opportunity! Earn Big Money Fast! Description: Are you tired of your 9-5 job and looking for a change? We have the perfect opportunity for you! Join our exclusive team and start making money from the comfort of your own home. Key Benefits: No Experience Required! Work Flexible Hours, Part-Time or Full-Time! Earn Up to $10,000 per Month! Quick and Easy Payments via Bitcoin! Requirements: Must be 18 years or older. No formal education needed. No background checks or verifications. Work-at-home kit available for a small fee. Perks: Free Accommodation Options! Travel Opportunities! Exciting Bonuses and Incentives! To apply, simply click the link below and register now! Dont miss out on this once-in-a-lifetime opportunity to change your life! SpammyLink123.com/ApplyNow
Enter the questions (comma-separated): What is the salary offered , Are there minors employed here, is accomodation offered


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
C:\Users\Apoorva Shetty\anaconda3\lib\site-packages\transformers\generation\utils.py:1281: UserWarning: Input length of input_ids is 36, but `max_length` is set to 36. This can lead to unexpected behavior. You should consider increasing `max_new_tokens`.
  warnings.warn(


RuntimeError: The expanded size of the tensor (36) must match the existing size (37) at non-singleton dimension 0.  Target sizes: [36].  Tensor sizes: [37]

In [12]:
#Authentication with HuggingFace
import os
os.environ["HUGGINGFACEHUB_API_TOKEN"] = HUGGINGFACEHUB_API_TOKEN

In [24]:
from langchain.llms import HuggingFaceHub

In [10]:
# Enter Hugging Face API Key
from getpass import getpass
HUGGINGFACEHUB_API_TOKEN = getpass()

········


In [25]:
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [18]:
question = "What arw the words associated with fradulent job offers"

template = """Question: {question}

Answer: Let's think step by step."""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [19]:
repo_id = "google/flan-t5-xxl" 

In [20]:
llm = HuggingFaceHub(
    repo_id=repo_id, model_kwargs={"temperature": 0.5, "max_length": 64}
)
llm_chain = LLMChain(prompt=prompt, llm=llm)

print(llm_chain.run(question))

C:\Users\Apoorva Shetty\anaconda3\lib\site-packages\huggingface_hub\utils\_deprecation.py:127: FutureWarning: '__init__' (from 'huggingface_hub.inference_api') is deprecated and will be removed from version '1.0'. `InferenceApi` client is deprecated in favor of the more feature-complete `InferenceClient`. Check out this guide to learn how to convert your script to use it: https://huggingface.co/docs/huggingface_hub/guides/inference#legacy-inferenceapi-client.
  warnings.warn(warning_message, FutureWarning)


Fradulent job offers are offers for employment that are not genuine. The words associated with fradulent job offers are fake, scam, false, and fraudulent. The answer: false.


In [30]:
from langchain.llms.huggingface_pipeline import HuggingFacePipeline